In [1]:
search_for = 136
start_from = 0
threads = 10
thread = 7
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.10128378868103027
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 1048119951
type: [1, 1, 1, 1, 136] 136
cases of this type: 342102016
100000 30.52288183276042
200000 48.630079503929224
300000 50.91143803508872
400000 52.79575789052087
500000 58.169377623891194
600000 55.50982848077207
700000 55.294619327376566
800000 54.43903427731658
900000 61.991430538813674
1000000 73.8310206638172
1100000 69.66887414083115
1200000 79.32878482200486
1300000 27.360203729686166
1400000 76.07404401735285
1500000 93.72014800951705
1600000 108.90891852639058
1700000 114.28940433440208
1800000 120.41340511705508
1900000 119.16882367499143
2000000 113.77392388766626
2100000 107.2137255887414
2200000 110.53881089036653
2300000 103.75882273920632
2400000 102.73310563185092
2500000 91.28307660852344
2600000 36.97030129278404
2700000 107.90752715751786
2800000 100.53060133321121
2900000 102.38377242707223
3000000 98.

29400000 105.74561918888783
29500000 104.67361320021793
29600000 105.31821819438781
29700000 108.30973814490807
29800000 108.65613462622292
29900000 106.33636439510016
30000000 70.89317659792697
30100000 85.50352099048173
30200000 114.02085759973045
30300000 103.69941100000452
30400000 92.1470959226099
30500000 57.93815392244072
30600000 107.95226578394826
30700000 104.74994427795374
30800000 104.00611348065797
30900000 102.8729580574118
31000000 112.40042591099474
31100000 106.85053666322416
31200000 100.84109823727343
31300000 100.86377376539477
31400000 100.79949829611516
31500000 19.76231143852032
31600000 103.75230200861338
31700000 98.07360750558563
31800000 98.32257961014334
31900000 103.47902224286655
32000000 109.43669093986495
32100000 100.91589287995653
32200000 101.48725226428508
32300000 108.56751623781217
32400000 103.47238802256749
32500000 65.30279296306547
32600000 84.76265736388166
32700000 109.90313332467746
32800000 109.22804316490753
32900000 112.30491782892179
330

59200000 49.62307123354795
59300000 97.29520223877036
59400000 97.01365271705424
59500000 99.48017301076241
59600000 98.91730834182056
59700000 102.58310332363816
59800000 102.2821987796882
59900000 102.15768682142512
60000000 37.274178325321216
60100000 116.02885875964178
60200000 101.55193676195513
60300000 105.12230667373845
60400000 110.65686058893701
60500000 104.78664255413176
60600000 100.96084643252355
60700000 107.90410012747836
60800000 89.18205195796415
60900000 53.27414941299719
61000000 99.25451739381691
61100000 100.05984600009027
61200000 94.21867246956579
61300000 99.22599160439519
61400000 90.5770473526277
61500000 92.98911025316352
61600000 86.63465926356447
61700000 29.34812791292609
61800000 94.02465542431766
61900000 96.53982696505038
62000000 91.35683699901533
62100000 94.42511586984415
62200000 95.35528324444032
62300000 95.81618200707892
62400000 104.22440766121362
62500000 33.17162731573701
62600000 109.3248113724939
62700000 97.43844312200603
62800000 105.4460

89400000 57.09811535259987
89500000 87.83618235804657
89600000 92.9429756356829
89700000 88.65532427648705
89800000 90.54988170306926
89900000 77.61973620835144
90000000 56.9283010143621
90100000 93.84497270148849
90200000 95.61054820128433
90300000 102.73229624804291
90400000 98.62618301449275
90500000 104.88460977208437
90600000 92.43628228066545
90700000 99.07295732650333
90800000 98.76681934665179
90900000 98.25809227925023
91000000 95.96952033184667
91100000 93.88263798723145
91200000 88.39140200760492
91300000 40.304210275846806
91400000 88.40683426336169
91500000 89.37234092254523
91600000 90.21874564774407
91700000 88.78271170766742
91800000 74.4001376839022
91900000 41.95894701132927
92000000 91.75748880622214
92100000 86.59781350474363
92200000 86.54999750739462
92300000 88.62534317655297
92400000 69.96338865629197
92500000 58.355573699886456
92600000 94.71274015383766
92700000 90.59938272131987
92800000 97.1239201806628
92900000 100.26094428480178
93000000 99.01131945081265


119000000 88.79208575643675
119100000 66.27202502059225
119200000 55.10061189194037
119300000 91.16999466322626
119400000 86.01737996793351
119500000 48.38624957113779
119600000 65.9405473925805
119700000 86.44072669685998
119800000 91.21315439374585
119900000 49.970488629010156
120000000 74.73584813430266
120100000 89.53080084586652
120200000 92.05419230407894
120300000 87.97541958797008
120400000 101.64647055099445
120500000 106.45424444671852
120600000 96.17952827167258
120700000 96.67396294595402
120800000 99.02373387107089
120900000 95.48105562321906
121000000 96.18418847120745
121100000 95.57444663580988
121200000 99.71530507225545
121300000 90.54641257893385
121400000 90.88166686621247
121500000 89.1457130235905
121600000 84.90896138981314
121700000 36.73367672821205
121800000 84.81532968122735
121900000 85.88699246351425
122000000 61.890677122677154
122100000 52.67101691802969
122200000 88.46476985168594
122300000 89.56313482964171
122400000 47.78799167885916
122500000 76.39214

...

953100000 2.21791253758593
953200000 1.5338878038301316
953300000 1.5860783585251523
953400000 0.9555353706237196
953500000 0.8593386572898197
953600000 1.7523601656903347
953700000 1.6972336255622247
953800000 1.5961199136002646
953900000 1.5542359030079265
954000000 1.121700543187221
954100000 0.6214717641291115
954200000 2.2829609497566516
954300000 1.6401707530034602
954400000 1.7028945929237744
954500000 1.5564423224455985
954600000 1.4232205812871515
954700000 0.5578500555152343
954800000 1.420087293771942
954900000 1.6509304821735542
best so far: 0
type: [1, 2, 1, 17, 4] 136
cases of this type: 2515456
955000000 1.8145069479821219
955100000 1.6164615668164135
955200000 1.4553490646005511
955300000 1.3206022670119404
955400000 1.2849508862036765
955500000 1.3220126290968337
955600000 1.2354079455623201
955700000 1.610763467994378
955800000 1.5004250956122895
955900000 1.2055362059708534
956000000 1.3732511258209599
956100000 1.1693415461680292
956200000 1.905114392379594
95630000

979200000 0.3354818787645538
979300000 0.36915633385999536
best so far: 0
type: [1, 2, 17, 2, 2] 136
cases of this type: 73984
979400000 0.337634608907685
best so far: 0
type: [1, 2, 17, 4, 1] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 1, 2] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 2, 1] 136
cases of this type: 18496
best so far: 0
type: [1, 2, 68, 1, 1] 136
cases of this type: 9248
979500000 0.3251997005029619
best so far: 0
type: [1, 4, 1, 1, 34] 136
cases of this type: 5345344
979600000 1.7287157360320853
979700000 1.8709720420177753
979800000 1.8936759964304606
979900000 1.367261923183736
980000000 1.4640836410298839
980100000 1.6929506951585271
980200000 1.5230962110885466
980300000 1.4537422429150322
980400000 1.3230614917327344
980500000 1.4777707951147574
980600000 1.3227197067606422
980700000 1.3728214514308892
980800000 1.6718797325904118
980900000 1.3163443104086658
981000000 1.585493272544547
981100000 1.6787504611029862
981200000 1.1

998700000 0.9438680322949846
998800000 1.3144760697970423
998900000 1.352107255015003
999000000 0.9154552700018784
999100000 1.1371749162556186
999200000 1.3706476946173283
999300000 1.244751204859507
999400000 1.0336507302047102
999500000 1.4543485789575155
999600000 1.6032303018386562
999700000 0.8763691143292435
999800000 1.6074582732117442
999900000 1.4259231920361917
1000000000 0.8152737553014456
1000100000 1.4041863514959687
1000200000 1.378849521028805
1000300000 0.9278845232244817
1000400000 1.4003518756368294
1000500000 0.25653411204336085
1000600000 0.25349347010997997
1000700000 0.2550607350657794
1000800000 0.2536908771060665
1000900000 0.251071945324813
1001000000 0.2517118477048185
1001100000 0.28724752355023625
1001200000 0.24531622047140733
1001300000 0.251090347722435
1001400000 0.25111456198510923
1001500000 0.2514886625598113
1001600000 0.3243730371407893
1001700000 0.250312473279649
1001800000 0.24896890099737443
1001900000 0.24820852946915228
1002000000 0.618990662

1025600000 0.3623899420343717
1025700000 0.27907114250412585
1025800000 0.25143464059912957
1025900000 0.34649593564216763
1026000000 0.3043276571135243
1026100000 0.23448095595757165
1026200000 0.35083284005596765
1026300000 0.3138188438394606
1026400000 0.17040656071354612
1026500000 0.35364418235312367
1026600000 0.3215983032095273
1026700000 0.1788345626859367
1026800000 0.34824261901047093
1026900000 0.31275107947455444
1027000000 0.2044892894639267
1027100000 0.31914577198555205
1027200000 0.32222743901892903
1027300000 0.22897438857119945
1027400000 0.29475559083387054
1027500000 0.33558397878117563
1027600000 0.2492638240951916
1027700000 0.2586958497023688
best so far: 0
type: [2, 1, 1, 17, 4] 136
cases of this type: 1257728
1027800000 0.3415570990424811
1027900000 0.26486197222232155
1028000000 0.24721073630565804
1028100000 0.26503152709919214
1028200000 0.2674967235276315
1028300000 0.23525649345697167
1028400000 0.25502426821083823
1028500000 0.2727039065122704
1028600000 

best so far: 0
type: [2, 34, 1, 2, 1] 136
cases of this type: 272
best so far: 0
type: [2, 34, 2, 1, 1] 136
cases of this type: 136
best so far: 0
type: [2, 68, 1, 1, 1] 136
cases of this type: 68
best so far: 0
type: [4, 1, 1, 1, 34] 136
cases of this type: 1336336
1045400000 0.04489748466562629
1045500000 0.05236506742131114
1045600000 0.04683338722852799
1045700000 0.04174429283606609
1045800000 0.046249022002526126
1045900000 0.03569513604144056
1046000000 0.028253249396991732
1046100000 0.01080585291595459
1046200000 0.030389544237931568
1046300000 0.03029942499261366
1046400000 0.03369388049357137
1046500000 0.02872272836876114
1046600000 0.027039010714583263
best so far: 0
type: [4, 1, 1, 2, 17] 136
cases of this type: 668168
1046700000 0.02875893809780081
1046800000 0.014982025046508179
1046900000 0.013994408546463648
1047000000 0.010914941393190622
1047100000 0.005459419867859947
1047200000 0.008785799422301186
1047300000 0.009348626504544417
best so far: 0
type: [4, 1, 1, 17,